<a href="https://colab.research.google.com/github/c-quilo/premiereDroplets/blob/main/AAEDropletExperiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Generative adversarial Networks (GANs) for droplet experiments

In [2]:
from __future__ import print_function, division

import tensorflow.keras as tf
import tensorflow
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as backend
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from keras import backend
from keras.layers import Lambda
from keras.constraints import Constraint
from keras.initializers import RandomNormal

from keras import optimizers
from keras.utils import np_utils
import tensorflow.keras as tf

The new class GANexperiments is defined

In [3]:
class ClipConstraint(Constraint):
    # set clip value when initialized
    def __init__(self, clip_value):
        self.clip_value = clip_value

    # clip model weights to hypercube
    def __call__(self, weights):
        return backend.clip(weights, -self.clip_value, self.clip_value)

    # get the config
    def get_config(self):
        return {'clip_value': self.clip_value}

# clip model weights to a given hypercube
class dropletGAN():

    def __init__(self, directory, trainingData, latent_dim, initNNodes, GANorWGAN):

        # Wasserstein loss
        def wasserstein_loss(y_true, y_pred):
            return backend.mean(y_true * y_pred)

        self.directory = directory
        self.trainingData = trainingData
        self.nFeatures = self.trainingData.shape[1]
        # Dimension of the latent space (noise)
        self.latent_dim = latent_dim
        self.constraint = 0.01
        self.dropoutNumber = 0.5
        self.alpha = 0.3
        self.GANorWGAN = GANorWGAN
        self.nameExperiment = '_dropletExperiments_'
        self.initNNodes = initNNodes

        self.c1_hist = []
        self.c2_hist = []
        self.g_hist = []

        self.optimizer = tf.optimizers.Nadam()

        if self.GANorWGAN == 'WGAN':
            self.loss = wasserstein_loss
        elif self.GANorWGAN == 'GAN':
            self.loss = 'binary_crossentropy'

        self.loss_gen = 'mse'

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()

        # Build the encoder and decoder
        self.generator_encoder = self.build_generator_encoder()
        self.generator_decoder = self.build_generator_decoder()

        # Only the generator is trained through the combined model, thus:
        self.discriminator.trainable = False

        # Connecting models
        real_input = tf.Input(shape=self.nFeatures)
        encoder_output = self.generator_encoder(real_input)
        decoder_output = self.generator_decoder(encoder_output)
        discriminator_output = self.discriminator(encoder_output)

        # The combined model stacks the autoencoder and discriminator
        # The stacked model has one input and two outputs: the decoded input and the discriminator output
        self.combined = tf.Model(real_input, [decoder_output, discriminator_output], name = 'AAE')
        self.combined.compile(loss=[self.loss_gen, self.loss], loss_weights=[0.999, 0.001], optimizer=self.optimizer)

    def build_discriminator(self):
        init = RandomNormal(stddev=0.02)
        const = ClipConstraint(0.01)

        in_disc = tf.Input(shape=(self.latent_dim))
        disc = tf.layers.LeakyReLU(self.alpha)(in_disc)
        disc = tf.layers.BatchNormalization()(disc)
        disc_output = tf.layers.Dense(1, activation='sigmoid')(disc)
        discriminator = tf.Model(in_disc, disc_output, name='Discriminator')
        discriminator.compile(loss=self.loss, optimizer=self.optimizer)

        return discriminator

    def build_generator_encoder(self):
        init = RandomNormal(stddev=0.02)
        init = tf.initializers.RandomNormal(stddev=0.02)

        input_enc = tf.Input(shape=self.nFeatures)
        nNodes = self.initNNodes
        flag = 0
        while nNodes > latent_dim:
            if flag == 0:
                enc = tf.layers.Dense(nNodes)(input_enc)
                flag = 1
            else:
                enc = tf.layers.Dense(nNodes)(enc)
            enc = tf.layers.LeakyReLU(self.alpha)(enc)
            enc = tf.layers.BatchNormalization()(enc)
            nNodes = nNodes / 2
        mu = tf.layers.Dense(latent_dim)(enc)
        sigma = tf.layers.Dense(latent_dim)(enc)

        # The latent representation ("fake") in a Gaussian distribution is then compared to the "real" arbitrary Gaussian
        # distribution fed in the Discriminator
        latent_repr = tf.layers.Lambda(
            lambda p: p[0] + backend.random_normal(backend.shape(p[0])) * backend.exp(p[1] / 2))(
            [mu, sigma])
        generator_encoder = tf.Model(input_enc, latent_repr, name='Encoder')
        generator_encoder.summary()
        return generator_encoder

    def build_generator_decoder(self):
        init = RandomNormal(stddev=0.02)
        init = tf.initializers.RandomNormal(stddev=0.02)

        # Input to the decoder is the latent space from the encoder
        input_dec = tf.Input(shape=self.latent_dim)
        n = 2 * latent_dim
        flag = 0
        while n <= self.initNNodes:
            if flag == 0:
                dec = tf.layers.Dense(n)(input_dec)
                flag = 1
            else:
                dec = tf.layers.Dense(n)(dec)
            dec = tf.layers.LeakyReLU(self.alpha)(dec)
            dec = tf.layers.BatchNormalization()(dec)
            n = n * 2
        output_dec = tf.layers.Dense(self.nFeatures, activation='tanh')(dec)
        generator_decoder = tf.Model(input_dec, output_dec, name='Decoder')

        generator_decoder.summary()
        return generator_decoder

    def train(self, epochs, batch_size=128, sample_interval=50, n_critic=5):

        # Load and pre process the data
        np.random.seed(42)
        global X_all
        data = self.trainingData
        
        min_ls = np.min(data)
        max_ls = np.max(data)
        min = -1
        max = +1

        def scaler(x, xmin, xmax, min, max):
            scale = (max - min) / (xmax - xmin)
            xScaled = scale * x + min - xmin * scale
            return xScaled

        X_all = scaler(data, min_ls, max_ls, min, max)

        if self.GANorWGAN == 'WGAN':
            real = -np.ones(batch_size)
            fake = np.ones(batch_size)

        if self.GANorWGAN == 'GAN':
            real = np.ones(batch_size)
            fake = np.zeros(batch_size)

        # Training the model
        for epoch in range(epochs):
            c1_tmp, c2_tmp = list(), list()

            # Training the discriminator more often than the generator
            for _ in range(n_critic):
                # Randomly selected samples and noise
                randomIndex = np.random.randint(0, X_all.shape[0], size=batch_size)
                noise = np.random.normal(0, 1, size=(batch_size, self.latent_dim))
                noise_training = np.random.normal(0, 0.1, size=(batch_size, self.nFeatures))
                # Select a random batch of input
                real_seqs = X_all[randomIndex] + noise_training

                # Generate a batch of new outputs (in the latent space) predicted by the encoder
                gen_seqs = self.generator_encoder.predict(real_seqs)

                # Train the discriminator
                # The arbitrary noise is considered to be a "real" sample
                d_loss_real = self.discriminator.train_on_batch(noise, real)
                c1_tmp.append(d_loss_real)
                # The latent space generated by the encoder is considered a "fake" sample
                d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
                c2_tmp.append(d_loss_fake)

            self.c1_hist.append(np.mean(c1_tmp))
            self.c2_hist.append(np.mean(c2_tmp))

            # Training the stacked model
            g_loss = self.combined.train_on_batch(real_seqs, [real_seqs, real])
            self.g_hist.append(g_loss)
            print("%d [C1 real: %f, C2 fake: %f], [G loss: %f, mse: %f]" % (epoch, self.c1_hist[epoch], self.c2_hist[epoch], g_loss[0], g_loss[1]))

            # Checkpoint progress: Plot losses and predicted data
            if epoch % sample_interval == 0:

                self.plot_loss(epoch)
                self.plot_values(epoch)
                self.generator_encoder.save(self.directory + '/' + 'encoder_' + self.nameExperiment + GANorWGAN +
                                            '_' + str(self.latent_dim) + '_' + str(epoch),
                                            save_format='tf')
                
                self.generator_decoder.save(self.directory + '/' + 'decoder_' + self.nameExperiment + GANorWGAN +
                                            '_' + str(self.latent_dim) + '_' + str(epoch),
                                            save_format='tf')

                self.discriminator.save(self.directory + '/' + 'discriminator_' + self.nameExperiment + GANorWGAN + 
                                        '_' + str(self.latent_dim) + '_' + str(epoch),
                                        save_format='tf')

    # Plots the (W)GAN related losses at every sample interval

    def plot_loss(self, epoch):
        fig = plt.figure()
        plt.plot(self.c1_hist, c='red')
        plt.plot(self.c2_hist, c='blue')
        plt.plot(self.g_hist[0][0], c='green')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title("GAN Loss per Epoch")
        plt.legend(['C real', 'C fake', 'Generator'])

        plt.savefig(self.directory + '/' + self.nameExperiment + GANorWGAN + '_losses_'  + str(epoch) +
                    '_' + str(self.latent_dim) + '.png')
        plt.close()

    # Plots predicted in the first 8 latent dimension at every sample interval

    def plot_values(self, epoch):
        fig = plt.figure(0, figsize=(20,10))
        noise = np.random.normal(0, 1, size=(X_all.shape[0]*100, self.latent_dim))
        prediction = self.generator_decoder(noise)
        nSamples = X_all.shape[0]
        sns.distplot(X_all[:])
        sns.distplot(prediction[:])

        plt.legend(['Ground truth', 'Generated data'])
        
        plt.tight_layout()
        plt.savefig(self.directory + '/' + self.nameExperiment + GANorWGAN + '_generated_data_' + str(epoch) +
                    '_' + str(self.latent_dim) + '.png')
        plt.close()

Execute the class and train the GAN

In [4]:
if __name__ == '__main__':
    #Load data
    directory = '/content/'
    filename = 'normalisedDropletData.npy'
    trainingData = np.load(directory + filename)
    epochs = 10000
    batch_size = 32
    n_critic = 5
    sample_interval = 1000
    initNNodes = 4
    latent_dim = 2

    #Training method
    GANorWGAN = 'WGAN'

    dropGAN = dropletGAN(directory = directory,
              trainingData=trainingData,
              latent_dim=latent_dim,
              initNNodes=initNNodes,
              GANorWGAN=GANorWGAN)
    dropGAN.train(epochs=epochs,
              batch_size=batch_size,
              sample_interval=sample_interval,
              n_critic = n_critic)

Model: "Encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 4)            36          input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 4)            0           dense_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 4)            16          leaky_re_lu_1[0][0]              
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


INFO:tensorflow:Assets written to: /content//encoder__dropletExperiments_WGAN_2_0/assets
INFO:tensorflow:Assets written to: /content//decoder__dropletExperiments_WGAN_2_0/assets
INFO:tensorflow:Assets written to: /content//discriminator__dropletExperiments_WGAN_2_0/assets
1 [C1 real: -0.502484, C2 fake: 0.501072], [G loss: 0.642324, mse: 0.643439]
2 [C1 real: -0.502700, C2 fake: 0.501434], [G loss: 0.631844, mse: 0.632967]
3 [C1 real: -0.501792, C2 fake: 0.501031], [G loss: 0.621973, mse: 0.623088]
4 [C1 real: -0.502226, C2 fake: 0.501114], [G loss: 0.583570, mse: 0.584598]
5 [C1 real: -0.502175, C2 fake: 0.501601], [G loss: 0.678490, mse: 0.679649]
6 [C1 real: -0.502227, C2 fake: 0.501541], [G loss: 0.603651, mse: 0.604761]
7 [C1 real: -0.502680, C2 fake: 0.501495], [G loss: 0.610100, mse: 0.611174]
8 [C1 real: -0.502723, C2 fake: 0.501072], [G loss: 0.512765, mse: 0.513772]
9 [C1 real: -0.502778, C2 fake: 0.501765], [G loss: 0.578859, mse: 0.579905]
10 [C1 real: -0.502564, C2 fake: 0

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


INFO:tensorflow:Assets written to: /content//encoder__dropletExperiments_WGAN_2_1000/assets
INFO:tensorflow:Assets written to: /content//decoder__dropletExperiments_WGAN_2_1000/assets
INFO:tensorflow:Assets written to: /content//discriminator__dropletExperiments_WGAN_2_1000/assets
1001 [C1 real: -0.389920, C2 fake: 0.258294], [G loss: 0.049133, mse: 0.049378]
1002 [C1 real: -0.392590, C2 fake: 0.295044], [G loss: 0.047758, mse: 0.047966]
1003 [C1 real: -0.374173, C2 fake: 0.277846], [G loss: 0.053865, mse: 0.054107]
1004 [C1 real: -0.397914, C2 fake: 0.299826], [G loss: 0.035360, mse: 0.035551]
1005 [C1 real: -0.393323, C2 fake: 0.301764], [G loss: 0.042264, mse: 0.042492]
1006 [C1 real: -0.391978, C2 fake: 0.318839], [G loss: 0.035941, mse: 0.036233]
1007 [C1 real: -0.391098, C2 fake: 0.233039], [G loss: 0.035834, mse: 0.036106]
1008 [C1 real: -0.408110, C2 fake: 0.285254], [G loss: 0.034041, mse: 0.034338]
1009 [C1 real: -0.389997, C2 fake: 0.264979], [G loss: 0.043818, mse: 0.044135

...

INFO:tensorflow:Assets written to: /content//encoder__dropletExperiments_WGAN_2_7000/assets
INFO:tensorflow:Assets written to: /content//decoder__dropletExperiments_WGAN_2_7000/assets
INFO:tensorflow:Assets written to: /content//discriminator__dropletExperiments_WGAN_2_7000/assets
7001 [C1 real: -0.060400, C2 fake: 0.038560], [G loss: 0.023592, mse: 0.023821]
7002 [C1 real: -0.067086, C2 fake: 0.043605], [G loss: 0.022431, mse: 0.022672]
7003 [C1 real: -0.070903, C2 fake: 0.018134], [G loss: 0.020884, mse: 0.021217]
7004 [C1 real: -0.063978, C2 fake: 0.031674], [G loss: 0.018516, mse: 0.018831]
7005 [C1 real: -0.062455, C2 fake: 0.028102], [G loss: 0.018415, mse: 0.018720]
7006 [C1 real: -0.057927, C2 fake: 0.036319], [G loss: 0.023743, mse: 0.023972]
7007 [C1 real: -0.072206, C2 fake: 0.014740], [G loss: 0.028802, mse: 0.029032]
7008 [C1 real: -0.061830, C2 fake: 0.036566], [G loss: 0.023125, mse: 0.023408]
7009 [C1 real: -0.065477, C2 fake: 0.024757], [G loss: 0.021910, mse: 0.022222

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


INFO:tensorflow:Assets written to: /content//encoder__dropletExperiments_WGAN_2_8000/assets
INFO:tensorflow:Assets written to: /content//decoder__dropletExperiments_WGAN_2_8000/assets
INFO:tensorflow:Assets written to: /content//discriminator__dropletExperiments_WGAN_2_8000/assets
8001 [C1 real: -0.073450, C2 fake: 0.045897], [G loss: 0.020295, mse: 0.020639]
8002 [C1 real: -0.052060, C2 fake: 0.028358], [G loss: 0.021017, mse: 0.021355]
8003 [C1 real: -0.068005, C2 fake: 0.031868], [G loss: 0.020932, mse: 0.021247]
8004 [C1 real: -0.083027, C2 fake: 0.038518], [G loss: 0.024467, mse: 0.024808]
8005 [C1 real: -0.070026, C2 fake: 0.022249], [G loss: 0.027668, mse: 0.027944]
8006 [C1 real: -0.084881, C2 fake: 0.014729], [G loss: 0.019613, mse: 0.019962]
8007 [C1 real: -0.060989, C2 fake: 0.030065], [G loss: 0.029379, mse: 0.029712]
8008 [C1 real: -0.081313, C2 fake: 0.025989], [G loss: 0.023454, mse: 0.023758]
8009 [C1 real: -0.057243, C2 fake: 0.034443], [G loss: 0.022096, mse: 0.022416

/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.7/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


INFO:tensorflow:Assets written to: /content//encoder__dropletExperiments_WGAN_2_9000/assets
INFO:tensorflow:Assets written to: /content//decoder__dropletExperiments_WGAN_2_9000/assets
INFO:tensorflow:Assets written to: /content//discriminator__dropletExperiments_WGAN_2_9000/assets
9001 [C1 real: -0.080412, C2 fake: 0.047229], [G loss: 0.021471, mse: 0.021830]
9002 [C1 real: -0.076807, C2 fake: 0.041032], [G loss: 0.021006, mse: 0.021416]
9003 [C1 real: -0.088513, C2 fake: 0.049294], [G loss: 0.022356, mse: 0.022709]
9004 [C1 real: -0.095586, C2 fake: 0.056615], [G loss: 0.017745, mse: 0.018149]
9005 [C1 real: -0.070240, C2 fake: 0.042226], [G loss: 0.025088, mse: 0.025500]
9006 [C1 real: -0.054038, C2 fake: 0.030016], [G loss: 0.026281, mse: 0.026656]
9007 [C1 real: -0.081198, C2 fake: 0.030990], [G loss: 0.023601, mse: 0.023874]
9008 [C1 real: -0.075500, C2 fake: 0.033999], [G loss: 0.020201, mse: 0.020621]
9009 [C1 real: -0.065991, C2 fake: 0.049512], [G loss: 0.020586, mse: 0.020958